# MPI v Pythonu = mpi4py
Knihovna mpi4py je Python rozhraní pro Message Passing Interface (MPI) standard, který je široce používán pro paralelní programování. MPI je komunikační protokol umožňující výměnu zpráv mezi procesy běžícími na různých procesorech nebo uzlech v distribuovaných nebo sdílených paměťových systémech.

Funkce mpi4py zahrnují:
1. Komunikaci mezi procesy pomocí zpráv.
2. Zasílání a přijímání dat mezi procesy.
3. Kolektivní operace, jako je broadcast, gather a reduce.
4. ... Mnoho dalších funkcí.

Nejprve však zprovoznění knihovny. 

Pro zprovoznění knihovny mpi4py na Linuxu je třeba provést vyjímečně dva kroky:

1. **Instalace MPI implementace**: Nejprve (pokud nemáte) je třeba nainstalovat nějakou MPI implementaci, například Open MPI nebo MPICH. Pro instalaci Open MPI na Ubuntu nebo Debianu, stačí následující příkaz: `sudo apt-get install -y openmpi-bin openmpi-common libopenmpi-dev`
2. **Instalace mpi4py**: Nainstalujte knihovnu mpi4py pomocí Pip: `pip3 install --user mpi4py`


Instalace MPI pod windows je standardně komplikovanější:

1. **Instalace MPI implementace**: Nejprve je třeba nainstalovat nějakou MPI implementaci, například Microsoft MPI (MS-MPI). Stáhněte si instalační balíčky pro MS-MPI z [oficiální stránky](https://docs.microsoft.com/en-us/message-passing-interface/microsoft-mpi) a nainstalujte je.
2. **Přidání MS-MPI do systémové proměnné PATH**: Ověřte, že cesty k `mpiexec.exe` a `mpiexec` jsou přidány do systémové proměnné PATH. Cesty by měly vypadat nějak takto: `C:\Program Files\Microsoft MPI\Bin\`.
3. Restartart systému ať se zaktualizuje path, a doufat, že to bude fungovat.



## Ukázka jednoduchého použití
Zde je jednoduchý příklad použití knihovny mpi4py:


In [ ]:
%%writefile mpi_example.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None

data = comm.scatter(data, root=0)

print(f"Proces {rank}: přijal {data} od procesu 0")


V tomto příkladu:
1. Importujeme knihovnu mpi4py.
2. Získáme komunikátor `comm`, který představuje skupinu procesů.
3. Zjistíme pořadí (rank) a počet procesů (size) v komunikátoru.
4. Proces s rankem 0 vytvoří seznam dat a rozešle jej ostatním procesům pomocí funkce `comm.scatter`.
5. Každý proces vypíše přijatá data.

Tento kód se spouští pomocí `mpirun` nástroje:

```bash
mpirun -n 4 python3 mpi_example.py
```

Kde `-n 4` značí počet procesů, které mají být spuštěny.

In [ ]:
!mpirun -n 4 python3 mpi_example.py

## Ukázka 2 posílání zpráv a polí
V tomto příkladu:

1. Importujeme knihovnu numpy a mpi4py.
2. Získáme komunikátor `comm`, rank a size.
3. Vytvoříme numpy pole `send_data` o velikosti 10 pro každý proces. Pole je naplněno hodnotou ranku.
4. Vytvoříme prázdné numpy pole `recv_data` pro přijetí dat.
5. Nastavíme `source` a `destination` pro každý proces podle ranku.
6. Použijeme metodu `Sendrecv` pro odeslání a přijetí dat mezi procesy.
7. Každý proces vypíše přijatá data.

In [ ]:
%%writefile mpi_example2.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Vytvoříme pole o velikosti 10 pro každý proces
send_data = np.full(10, rank, dtype=int)
recv_data = np.zeros(10, dtype=int)

# Proces 0 získá data od procesu 1, proces 1 od procesu 2 atd.
# Poslední proces získá data od procesu 0
source = rank - 1 if rank > 0 else size - 1
destination = (rank + 1) % size

# Procesy si předají data pomocí sendrecv
comm.Sendrecv(send_data, dest=destination, recvbuf=recv_data, source=source)

print(f"Proces {rank}: Přijal data {recv_data} od procesu {source}")

In [ ]:
!mpirun -n 4 python3 mpi_example2.py